In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/github/selfiespredict


/content/drive/My Drive/github/selfiespredict


In [13]:
%ls

gpu.log  gpu_usage.sh  run/  test_model_params.yaml


In [36]:
!git pull 

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 10 (delta 4), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (10/10), done.
From https://github.com/bananenpampe/selfiespredict
   96c9a51..20e03c9  main       -> origin/main
Updating 96c9a51..20e03c9
Fast-forward
 run/testmodel/run.yaml               | 23 +++++++++++++++++++++++
 run/testmodel/test_model_params.yaml |  1 +
 2 files changed, 24 insertions(+)
 create mode 100644 run/testmodel/run.yaml


In [ ]:
! pip list

In [6]:
! pip install -e .

Obtaining file:///content/drive/My%20Drive/github/selfiespredict
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 216 kB 4.3 MB/s 
     |████████████████████████████████| 20.6 MB 1.4 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 16.0 MB 12.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
  Running setup.py develop for selfiespredict


In [4]:
%cd /content/drive/My Drive/github/selfiespredict/run/testmodel

/content/drive/My Drive/github/selfiespredict/run/testmodel


In [8]:
%cd run/

/content/drive/My Drive/github/selfiespredict/run/testmodel/run


In [12]:
%cd ..

/content/drive/My Drive/github/selfiespredict/run/testmodel


In [25]:
! onmt_build_vocab -config test_model_params.yaml -src_seq_length 3000 -tgt_seq_length 3000 -src_vocab_size 3000 -tgt_vocab_size 3000 

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-11-23 21:25:52,682 INFO] Counter vocab from 5000 samples.
[2021-11-23 21:25:52,682 INFO] Build vocab on 5000 transformed examples/corpus.
[2021-11-23 21:25:52,694 INFO] corpus_1's transforms: TransformPipe()
[2021-11-23 21:25:52,901 INFO] Counters src:141
[2021-11-23 21:25:52,901 INFO] Counters tgt:51
[2021-11-23 21:25:52,901 INFO] Counters after share:142
[2021-11-23 21:25:52,902 WARNING] path ./run/example.vocab.src exists, may overwrite...


In [28]:
%%writefile gpu_usage.sh
#! /bin/bash
#comment: run for 10 seconds, change it as per your use
end=$((SECONDS+100))

while [ $SECONDS -lt $end ]; do
    nvidia-smi --format=csv --query-gpu=power.draw,utilization.gpu,memory.used,memory.free,fan.speed,temperature.gpu >> gpu.log
    #comment: or use below command and comment above using #
    #nvidia-smi dmon -i 0 -s mu -d 1 -o TD >> gpu.log
done

Overwriting gpu_usage.sh


In [20]:
%cd run
%ls

/content/drive/My Drive/github/selfiespredict/run/testmodel/run
example.vocab.src


In [21]:
%cd ..

/content/drive/My Drive/github/selfiespredict/run/testmodel


In [33]:
%ls

gpu.log  gpu_usage.sh  run/  test_model_params.yaml


In [19]:
!head -n 10 test_model_params.yaml

# test_model_params.yaml

## Where the samples will be written
save_data: ./run/example
## Where the vocab(s) will be written, should point to same in shared embeddings
src_vocab: ./run/example.vocab.src
tgt_vocab: ./run/example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: False



In [3]:
%ls

drive/  sample_data/


In [10]:
%%bash --bg

bash gpu_usage.sh

Starting job # 4 in a separate thread.


In [ ]:
! onmt_train -config run.yaml \
        -seed 42 -gpu_ranks 0  \
        -train_steps 250000 -param_init 0 \
        -param_init_glorot -max_generator_batches 32 \
        -batch_type tokens -batch_size 6144\
         -normalization tokens -max_grad_norm 0  -accum_count 4 \
        -optim adam -adam_beta1 0.9 -adam_beta2 0.998 -decay_method noam  \
        -warmup_steps 8000 -learning_rate 2 -label_smoothing 0.0 \
        -layers 4 -rnn_size  384 -word_vec_size 384 \
        -encoder_type transformer -decoder_type transformer \
        -dropout 0.1 -position_encoding -share_embeddings  \
        -global_attention general -global_attention_function softmax \
        -self_attn_type scaled-dot -heads 8 -transformer_ff 2048 \
        -report_every 2500

[2021-11-23 22:33:26,323 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-11-23 22:33:26,324 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-11-23 22:33:26,325 INFO] Missing transforms field for valid data, set to default: [].
[2021-11-23 22:33:26,325 INFO] Parsed 2 corpora from -data.
[2021-11-23 22:33:26,326 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-11-23 22:33:26,326 INFO] Loading vocab from text file...
[2021-11-23 22:33:26,326 INFO] Loading src vocabulary from ./run/example.vocab.src
[2021-11-23 22:33:26,327 INFO] Loaded src vocab has 142 tokens.
[2021-11-23 22:33:26,327 INFO] Loading tgt vocabulary from ./run/example.vocab.src
[2021-11-23 22:33:26,329 INFO] Loaded tgt vocab has 142 tokens.
[2021-11-23 22:33:26,329 INFO] Building fields with vocab in counters...
[2021-11-23 22:33:26,330 INFO]  * tgt vocab size: 146.
[2021-11-23 22:33:26,330 INFO]  * src vocab size: 144.
[2021-11-23 

In [ ]:
#8791/5579 tok/s

In [27]:
! onmt_train -config test_model_params.yaml -report_every 5000 

[2021-11-23 21:40:50,377 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-11-23 21:40:50,378 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-11-23 21:40:50,378 INFO] Missing transforms field for valid data, set to default: [].
[2021-11-23 21:40:50,379 INFO] Parsed 2 corpora from -data.
[2021-11-23 21:40:50,380 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-11-23 21:40:50,380 INFO] Loading vocab from text file...
[2021-11-23 21:40:50,380 INFO] Loading src vocabulary from ./run/example.vocab.src
[2021-11-23 21:40:50,381 INFO] Loaded src vocab has 142 tokens.
[2021-11-23 21:40:50,382 INFO] Loading tgt vocabulary from ./run/example.vocab.src
[2021-11-23 21:40:50,383 INFO] Loaded tgt vocab has 142 tokens.
[2021-11-23 21:40:50,383 INFO] Building fields with vocab in counters...
[2021-11-23 21:40:50,383 INFO]  * tgt vocab size: 146.
[2021-11-23 21:40:50,384 INFO]  * src vocab size: 144.
[2021-11-23 

In [22]:
%cd ..

/content/drive/My Drive/github/selfiespredict/run/testmodel


In [7]:
%ls testmodel

test_model_params.yaml


In [ ]:
cleaner = Data_Cleaner()

In [ ]:
cleaner.import_data("USPTO_STEREO")

Downloading...
From: https://drive.google.com/uc?id=1r3_7WMEor7-CgN34Foj-ET-uFco0fURU
To: /content/drive/My Drive/github/selfiespredict/data/raw_data/USPTO_STEREO/src-train.txt
100%|██████████| 137M/137M [00:01<00:00, 123MB/s]


/content/drive/My Drive/github/selfiespredict/data/raw_data/USPTO_STEREO/tgt-train.txt exists, skip downloading
/content/drive/My Drive/github/selfiespredict/data/raw_data/USPTO_STEREO/src-val.txt exists, skip downloading
/content/drive/My Drive/github/selfiespredict/data/raw_data/USPTO_STEREO/tgt-val.txt exists, skip downloading
/content/drive/My Drive/github/selfiespredict/data/raw_data/USPTO_STEREO/src-test.txt exists, skip downloading
/content/drive/My Drive/github/selfiespredict/data/raw_data/USPTO_STEREO/tgt-test.txt exists, skip downloading


In [ ]:
cleaner.gen_txt("USPTO_480k")

KeyboardInterrupt: ignored

In [ ]:
import numpy as np

In [ ]:
%mkdir pip_fix

mkdir: cannot create directory ‘pip_fix’: File exists


In [ ]:
username = ''
repository = ''
git_token = ''
!git clone https://{git_token}@github.com/{username}/{repository}

Cloning into 'selfiespredict'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 195 (delta 82), reused 170 (delta 65), pack-reused 0
Receiving objects: 100% (195/195), 22.60 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Checking out files: 100% (56/56), done.
